In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from typing import Dict, Tuple
import keras.layers as l
from keras import models, callbacks, utils, losses
import string
import re

2024-05-26 13:18:53.597796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 13:18:54.362425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
valid_letters = set("ёйцукенгшщзхъфывапролджэячсмитьбюЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ" + string.printable)
dirname = "books"

text = []
for subdir in os.listdir(dirname):
    for text_file in os.listdir(f"{dirname}/{subdir}"):
        if not re.match(r"[a-zA-Z_0-9]+.txt", text_file):
            continue
        with open(f"{dirname}/{subdir}/{text_file}") as f:
            text.extend([ch for ch in f.read() if ch in valid_letters])
text = "".join(text)[:2_000_000]

In [3]:
def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[char] for char in text])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)


2024-05-26 13:22:11.597161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79262 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:8c:00.0, compute capability: 8.0


In [4]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    keras.layers.LSTM(512, return_sequences=True, stateful=True),
    keras.layers.LSTM(512, return_sequences=True, stateful=True),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=65, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

2024-05-26 13:22:12.758319: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-26 13:22:12.760134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-26 13:22:12.761332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/65


2024-05-26 13:22:13.267236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-26 13:22:13.268993: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-26 13:22:13.270808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

122/122 [==============================] - 8s 46ms/step - loss: 3.2171 - accuracy: 0.1650
Epoch 2/65
122/122 [==============================] - 2s 19ms/step - loss: 2.6031 - accuracy: 0.2533
Epoch 3/65
122/122 [==============================] - 2s 17ms/step - loss: 2.3453 - accuracy: 0.3155
Epoch 4/65
122/122 [==============================] - 3s 21ms/step - loss: 2.1477 - accuracy: 0.3694
Epoch 5/65
122/122 [==============================] - 3s 21ms/step - loss: 1.9797 - accuracy: 0.4130
Epoch 6/65
122/122 [==============================] - 2s 17ms/step - loss: 1.8449 - accuracy: 0.4490
Epoch 7/65
122/122 [==============================] - 2s 16ms/step - loss: 1.7399 - accuracy: 0.4767
Epoch 8/65
122/122 [==============================] - 2s 17ms/step - loss: 1.6588 - accuracy: 0.4981
Epoch 9/65
122/122 [==============================] - 3s 23ms/step - loss: 1.5963 - accuracy: 0.5146
Epoch 10/65
122/122 [==============================] - 3s 23ms/step - loss: 1.5465 - accuracy: 0.5278


In [5]:
def predict_next(sample: str, model: keras.Sequential, tokenizer: Dict[str, int], vocabulary: Dict[int, str], n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [9]:
print(predict_next(
    sample='что',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=200,
    temperature=0.9,
    batch_size=BATCH_SIZE
))

что-нибудь в тот дело, когда-нибудь страх? - Мне с ним поднимаются. Теперь уже не хотелось. Он собрал объединиваться на две горячего голово?.. - С десять циновой проектов, давай еред отраженное разговору


In [10]:
print(predict_next(
    sample='где',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=100,
    temperature=0.2,
    batch_size=BATCH_SIZE
))

где бандитская глаза - просто он просто стоит запотевшего старые рассуждения ничего не выходит на две п


In [11]:
print(predict_next(
    sample='когда',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=150,
    temperature=0.9,
    batch_size=BATCH_SIZE
))

когда из-за лампочка, хотя и не позвонил вам, - его терзала. Я бы тоже предполагал, одновременно скользил головой, больше ничего. - Нет, постой, одно двена
